In [1]:
import numpy as np
import random

class Conv3x3:
    #a convolutin layer using 3x3 filters

    def __init__(self, num_filters):
        self.num_filters = num_filters
        #filter initialization
        self.filters = np.random.randn(num_filters,3,3) / 9
    
    '''
    
    '''
    def iterate_(self, image):
        row, col = image.shape
        #conv layers and output layer initialization
        self.layer = np.zeros(())

        for i in range(row):
            for j in range(col):
                if i <= row-2 and j <= col-2: 
                    roi = image[i:(i+3),j:(j+3)]
                    yield roi, i, j
    
    def forward(self, image):
        row, col = image.shape
        #conv layers and output layer initialization
        self.layer = np.zeros((self.num_filters, row-2, col-2))

        for 
        


In [3]:
class Conv3x3:
  def __init__(self, num_filters):
    self.num_filters = num_filters

    # filters is a 3d array with dimensions (num_filters, 3, 3)
    # We divide by 9 to reduce the variance of our initial values
    self.filters = np.random.randn(num_filters, 3, 3) / 9

  def iterate_regions(self, image):
    '''
    Generates all possible 3x3 image regions using valid padding.
    - image is a 2d numpy array
    '''
    h, w = image.shape

    for i in range(h - 2):
      for j in range(w - 2):
        im_region = image[i:(i + 3), j:(j + 3)]
        yield im_region, i, j

  def forward(self, input):
    '''
    Performs a forward pass of the conv layer using the given input.
    Returns a 3d numpy array with dimensions (h, w, num_filters).
    - input is a 2d numpy array
    '''
    h, w = input.shape
    output = np.zeros((h - 2, w - 2, self.num_filters))

    for im_region, i, j in self.iterate_regions(input):
        # print(im_region * self.filters)
        # if i == 1 and j == 1:
            # print(im_region)
            # print(self.filters)
            # print(im_region*self.filters)
        output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
        # print(output[i,j])

    return output

In [4]:
from tensorflow.keras import datasets, layers, models
import keras
import tensorflow as tf

In [5]:
((train_data, train_labels),
 (test_data, test_labels)) = tf.keras.datasets.fashion_mnist.load_data()

In [11]:
conv = Conv3x3(8)
output = conv.forward(train_data[0])
print(conv.filters)
print(output.shape)
# print(output[:,:,0])

[[[ 0.11365985 -0.00758996  0.12630547]
  [ 0.12877869  0.15931339 -0.05127237]
  [ 0.08223241  0.05012807  0.05971327]]

 [[ 0.02996919 -0.25995347  0.08653329]
  [-0.06105064  0.0010477  -0.07142494]
  [ 0.12119641 -0.12035447  0.03593307]]

 [[ 0.07722197 -0.06215154  0.12571277]
  [-0.1407588   0.10030161 -0.02208413]
  [ 0.03705418 -0.0637095  -0.03065436]]

 [[ 0.0342773  -0.09168923 -0.02226484]
  [-0.2025426   0.15166165 -0.03508157]
  [ 0.12703071  0.11189797  0.0228812 ]]

 [[-0.0082654  -0.00712442 -0.03926309]
  [-0.02550814  0.1101267   0.30616133]
  [-0.10440372  0.00777137 -0.15876623]]

 [[ 0.03649871  0.01974442 -0.0905324 ]
  [-0.14778947  0.16349882  0.10221842]
  [ 0.10642636 -0.10048099  0.08522247]]

 [[-0.08614458  0.14926506 -0.03439336]
  [ 0.12087436 -0.05534786  0.26505823]
  [-0.07997474  0.00077016  0.11320699]]

 [[-0.06412859 -0.00306926  0.18540478]
  [-0.09565002  0.03972184 -0.07389816]
  [ 0.01753752 -0.03541562  0.13278329]]]
(26, 26, 8)


In [18]:
class MaxPool2:
    def iterate_regions(self,image):
       h,w,_ = image.shape
       new_h = h//2
       new_w = w//2

       for i in range(0,w,2):
           for j in range(0,h,2):
               im_region = image[i:(i+2),j:(j+2)]
               yield im_region, i//2 ,j//2

    def forward(self, input):
        h,w,num_filters = input.shape
        output = np.zeros((h//2,w//2,num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i,j] = np.amax(im_region, axis=(0,1))

        return output 

In [19]:
pool = MaxPool2()
output = pool.forward(output)
print(output.shape)

(13, 13, 8)


In [27]:
class Softmax:
  # A standard fully-connected layer with softmax activation.

  def __init__(self, input_len, nodes):
    # We divide by input_len to reduce the variance of our initial values
    self.weights = np.random.randn(input_len, nodes) / input_len
    self.biases = np.zeros(nodes)

  def forward(self, input):
    '''
    Performs a forward pass of the softmax layer using the given input.
    Returns a 1d numpy array containing the respective probability values.
    - input can be any array with any dimensions.
    '''
    self.last_input_shape = input.shape

    input = input.flatten()
    self.last_input = input

    input_len, nodes = self.weights.shape

    totals = np.dot(input, self.weights) + self.biases
    self.last_totals = totals
    
    exp = np.exp(totals)
    return exp / np.sum(exp, axis=0)

In [24]:
import mnist
test_images = mnist.test_images()[:1000]
test_labels = mnist.test_labels()[:1000]

In [25]:
conv = Conv3x3(8)                  # 28x28x1 -> 26x26x8
pool = MaxPool2()                  # 26x26x8 -> 13x13x8
softmax = Softmax(13 * 13 * 8, 10) # 13x13x8 -> 10

In [26]:
def forward(image, label):
  '''
  Completes a forward pass of the CNN and calculates the accuracy and
  cross-entropy loss.
  - image is a 2d numpy array
  - label is a digit
  '''
  # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
  # to work with. This is standard practice.
  out = conv.forward((image / 255) - 0.5)
  out = pool.forward(out)
  out = softmax.forward(out)

  # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
  loss = -np.log(out[label])
  acc = 1 if np.argmax(out) == label else 0

  return out, loss, acc

print('MNIST CNN initialized!')

loss = 0
num_correct = 0
for i, (im, label) in enumerate(zip(test_images, test_labels)):
  # Do a forward pass.
  _, l, acc = forward(im, label)
  loss += l
  num_correct += acc

  # Print stats every 100 steps.
  if i % 100 == 99:
    print(
      '[Step %d] Past 100 steps: Average Loss %.3f | Accuracy: %d%%' %
      (i + 1, loss / 100, num_correct)
    )
    loss = 0
    num_correct = 0

MNIST CNN initialized!
[Step 100] Past 100 steps: Average Loss 2.303 | Accuracy: 3%
[Step 200] Past 100 steps: Average Loss 2.303 | Accuracy: 8%
[Step 300] Past 100 steps: Average Loss 2.304 | Accuracy: 4%
[Step 400] Past 100 steps: Average Loss 2.303 | Accuracy: 7%
[Step 500] Past 100 steps: Average Loss 2.303 | Accuracy: 4%
[Step 600] Past 100 steps: Average Loss 2.303 | Accuracy: 6%
[Step 700] Past 100 steps: Average Loss 2.303 | Accuracy: 5%
[Step 800] Past 100 steps: Average Loss 2.303 | Accuracy: 7%
[Step 900] Past 100 steps: Average Loss 2.303 | Accuracy: 4%
[Step 1000] Past 100 steps: Average Loss 2.304 | Accuracy: 5%
